In [1]:
# add to sys.path
import sys

sys.path.insert(0, "../")
sys.path.insert(0, "../src")

import time
import logging
import pprint
import pandas as pd
from datetime import datetime, timedelta, timezone
from dateutil.relativedelta import relativedelta
import plaid

# gcp
from google.cloud import bigquery

# SCHEMAS
from schemas.bq_table_schemas import BqTableSchemas
from schemas.cloud_schemas import CloudSchemas

# UTILS
from utils.bq_utils import BqUtils
from utils.google_cloud_utils import GcpUtils
from utils.plaid_utils import PlaidUtils
from utils.secrets_utils import SecretsUtils
from utils.crypto_utils import CryptoUtils
from utils.sendgrid_utils import SendgridUtils

# JOBS
from jobs.budget_values import BudgetValues
from jobs.financial_accounts import FinancialAccounts
from jobs.plaid_transactions import PlaidTransactions
from jobs.plaid_investments import PlaidInvestments

# get all secrets
sec = SecretsUtils()
secrets = sec.create_secrets_dict(plaid_secret_type="PROD")

PLAID_CLIENT_ID = secrets["PLAID_CLIENT_ID"]
PLAID_SECRET = secrets["PLAID_SECRET_PROD"]
PLAID_ACCESS_TOKENS = sec.get_access_token_secrets(secrets)
PLAID_HOST = plaid.Environment.Production
PLAID_PRODUCTS = ["liabilities", "transactions", "investments"]
PLAID_COUNTRY_CODES = ["US"]
CRYPTO_SECRETS = secrets["CRYPTO_SECRETS"]
SENDGRID_KEY = secrets["SENDGRID_KEY"]

# initialize clients
bq_client = bigquery.Client()
bq = BqUtils(bq_client=bq_client)
plaid_client = PlaidUtils(bq_client, PLAID_CLIENT_ID, PLAID_SECRET, PLAID_HOST)
financial_accounts = FinancialAccounts(bq_client, plaid_client)
plaid_transactions = PlaidTransactions(bq_client, plaid_client)
plaid_investments = PlaidInvestments(bq_client, plaid_client)
budget_values = BudgetValues(bq_client)
crypto = CryptoUtils()
bq_tables = BqTableSchemas()
gcp = GcpUtils(bq_client=bq_client)
sendgrid = SendgridUtils(SENDGRID_KEY)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 1000)

In [2]:
df1 = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6]
})

df2 = pd.DataFrame({
    'X': [10, 20, 30],
    'Y': [40, 50, 60]
})

In [3]:
# message_1 = sendgrid.create_html_message_with_pandas_df("Testing 1st message", df1)
# message_2 = sendgrid.create_html_message_with_pandas_df("Testing 2nd message", df2)
# message_content = sendgrid.chain_html_messages([message_1, message_2])
# email_message = sendgrid.construct_email_message(
#     from_email="zach.s.cox@gmail.com",
#     to_emails=["zach.s.cox+zsc-alerts@gmail.com"],
#     email_subject="ZSC Alerts",
#     html_message=message_content
# )

# sendgrid.send_email(email_message)

# # sendgrid.get_email_content(email_message)